In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/09/21 09:53:45 WARN Utils: Your hostname, DESKTOP-RKUV3OS resolves to a loopback address: 127.0.1.1; using 172.23.74.72 instead (on interface eth0)
22/09/21 09:53:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/21 09:53:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Import Table

In [4]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'split_tags.parquet', '../data/merchants_preprocess/', truncate=20)

================ Merchant ================
|> Loading File...


|> Loading Finished!
-RECORD 0-----------------------------
 merchant_name | Felis Limited        
 tags          | furniture, home f... 
 merchant_abn  | 10023283211          
 take_rate     | 0.18                 
 type          | e                    
only showing top 1 row



In [5]:
print("================ Transactions ================")
transactions_sdf = read_file(spark, 'transactions_20210228_20210827_snapshot')

================ Transactions ================
|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 18478                                
 merchant_abn   | 62191208634                          
 dollar_value   | 63.255848959735246                   
 order_id       | 949a63c8-29f7-4ab0-ada4-99ac50a88952 
 order_datetime | 2021-08-20                           
only showing top 1 row



In [8]:
temp_sdf = consumer_sdf.join(user_consumer_conn_sdf, how='left', on='consumer_id')
temp_sdf = temp_sdf.withColumnRenamed('name', 'consumer_name')

NameError: name 'consumer_sdf' is not defined

In [9]:
rrm_sdf = read_file(spark, 'external_rrm.parquet', '../data/curated/', truncate=20)

|> Loading File...
|> Loading Finished!
-RECORD 0-------------------
 Date | 2020-02-21 11:00:00 
 AUS  | 1                   
 NSW  | 1                   
 VIC  | 1                   
 QLD  | -1                  
 SA   | 2                   
 WA   | 2                   
 TAS  | -1                  
 NT   | 4                   
 ACT  | 4                   
only showing top 1 row

